In [1]:
import autorootcwd
import torch
from audiocraft.models import MusicGen

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.4.1 with CUDA None (you have 2.1.0)
    Python  3.8.10 (you have 3.8.10)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/Users/pierrelardet/.pyenv/versions/UG4Diss/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = MusicGen.get_pretrained("facebook/musicgen-small")  # or "medium", "melody", etc.
# model.eval()  # set to evaluation mode
# model = model.cuda()  # if you want CUDA acceleration
# Get mps device

/Users/pierrelardet/.pyenv/versions/UG4Diss/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [26]:
import torchaudio

wav, sr = torchaudio.load("./data/processed/audio/bingcrosby_silentnight.mp3")
new_sr = 32000
resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=new_sr)
wav = resampler(wav)
assert model.sample_rate == new_sr, "Model sample rate must match input sample rate"
print(wav.shape, new_sr)

torch.Size([2, 5183146]) 32000


In [27]:
wav = wav.unsqueeze(0)  # add batch dimension
if wav.shape[1] > 1:
    wav = wav.mean(dim=1, keepdim=True)  # convert stereo to mono if needed
wav.shape

torch.Size([1, 1, 5183146])

In [ ]:
with torch.no_grad():
    # model.compression_model.encode returns (tokens, scale)
    prompt_tokens, _ = model.compression_model.encode(wav)
    # Note: prompt_tokens is typically a tensor of shape (batch, token_length)
    # For MusicGen, these tokens may represent multiple codebooks interleaved.
    print("Encoded tokens shape:", prompt_tokens.shape)


In [21]:
wav.shape

torch.Size([1, 1])

In [11]:
# audio_encoder = model.compression_model

# with torch.no_grad():
#     encoded_frames = audio_encoder.encode(wav)  # encode the audio
#     codes = encoded_frames[0].codes

In [4]:
from transformers import EncodecModel, AutoProcessor

encoder = EncodecModel.from_pretrained("facebook/encodec_48khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_48khz")

/Users/pierrelardet/.pyenv/versions/UG4Diss/lib/python3.8/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [5]:
# Encode audio

inputs = processor(wav, return_tensors="pt", padding="longest", sampling_rate=48000)


In [6]:
inputs['input_values'].shape

torch.Size([1, 2, 7793760])

In [8]:
encoder_inputs = encoder.encode(inputs['input_values'], inputs['padding_mask'])

In [14]:
encoder_inputs['audio_codes'].shape

torch.Size([164, 1, 2, 150])